In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df = pd.read_excel("Online Retail.xlsx")
df.head()
df1 = df

In [3]:
df1.Country.nunique()

38

In [4]:
customer_country=df1[['Country','CustomerID']].drop_duplicates()
customer_country.groupby(['Country'])['CustomerID'].aggregate('count').reset_index().sort_values('CustomerID', ascending=False)

,Country,CustomerID
36,United Kingdom,3950
14,Germany,95
13,France,87
31,Spain,31
3,Belgium,25
33,Switzerland,21
27,Portugal,19
19,Italy,15
12,Finland,12
1,Austria,11


In [6]:
df1 = df1.loc[df1['Country'] == 'United Kingdom']

In [8]:
df1.isnull().sum(axis=0)

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133600
Country             0
dtype: int64

In [10]:
df1 = df1[pd.notnull(df1['CustomerID'])]

In [11]:
df1 = df1[(df1['Quantity']>0)]
df1.shape
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354345 entries, 0 to 541893
Data columns (total 8 columns):
InvoiceNo      354345 non-null object
StockCode      354345 non-null object
Description    354345 non-null object
Quantity       354345 non-null int64
InvoiceDate    354345 non-null datetime64[ns]
UnitPrice      354345 non-null float64
CustomerID     354345 non-null float64
Country        354345 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 24.3+ MB


In [12]:
def unique_counts(df1):
   for i in df1.columns:
       count = df1[i].nunique()
       print(i, ": ", count)
unique_counts(df1)

InvoiceNo :  16649
StockCode :  3645
Description :  3844
Quantity :  294
InvoiceDate :  15615
UnitPrice :  403
CustomerID :  3921
Country :  1


In [13]:
df1['TotalPrice'] = df1['Quantity'] * df1['UnitPrice']

In [120]:
import datetime as dt
NOW = dt.datetime(2011,12,10)


In [71]:
df1['InvoiceDate'] = pd.to_datetime(df1['InvoiceDate'])

In [166]:
rfmTable = df1.groupby('CustomerID',as_index=False).agg({'InvoiceDate': ['max'], # Recency
                                        'InvoiceNo': ['count'],      # Frequency
                                        'TotalPrice': ['sum']})
rfmTable.columns = rfmTable.columns.droplevel(level=0)
rfmTable['recency']=(NOW-rfmTable['max']).dt.days


In [168]:
rfmTable.rename(columns={'':'customer_id',
                         'count': 'frequency', 
                         'sum': 'monetary_value'}, inplace=True)

In [169]:
rfmTable.head()

,customer_id,max,monetary_value,frequency,recency
0,12346.0,2011-01-18 10:01:00,77183.60,1,325
1,12747.0,2011-12-07 14:34:00,4196.01,103,2
2,12748.0,2011-12-09 12:20:00,33719.73,4596,0
3,12749.0,2011-12-06 09:56:00,4090.88,199,3
4,12820.0,2011-12-06 15:12:00,942.34,59,3


In [170]:

quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles

,customer_id,monetary_value,frequency,recency
0.25,14208.0,300.04,17.0,17.0
0.50,15569.0,651.82,41.0,50.0
0.75,16913.0,1575.89,99.0,142.0


In [172]:

quantiles = quantiles.to_dict()
quantiles

{'customer_id': {0.25: 14208.0, 0.5: 15569.0, 0.75: 16913.0},
 'frequency': {0.25: 17.0, 0.5: 41.0, 0.75: 99.0},
 'monetary_value': {0.25: 300.03999999999996,
  0.5: 651.82000000000016,
  0.75: 1575.8900000000003},
 'recency': {0.25: 17.0, 0.5: 50.0, 0.75: 142.0}}

In [173]:
segmented_rfm = rfmTable

In [174]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [175]:
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))

In [176]:
segmented_rfm.head()

,customer_id,max,monetary_value,frequency,recency,r_quartile,f_quartile,m_quartile
0,12346.0,2011-01-18 10:01:00,77183.60,1,325,4,4,1
1,12747.0,2011-12-07 14:34:00,4196.01,103,2,1,1,1
2,12748.0,2011-12-09 12:20:00,33719.73,4596,0,1,1,1
3,12749.0,2011-12-06 09:56:00,4090.88,199,3,1,1,1
4,12820.0,2011-12-06 15:12:00,942.34,59,3,1,2,2


In [177]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) \
                            + segmented_rfm.f_quartile.map(str) \
                            + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,customer_id,max,monetary_value,frequency,recency,r_quartile,f_quartile,m_quartile,RFMScore
0,12346.0,2011-01-18 10:01:00,77183.60,1,325,4,4,1,441
1,12747.0,2011-12-07 14:34:00,4196.01,103,2,1,1,1,111
2,12748.0,2011-12-09 12:20:00,33719.73,4596,0,1,1,1,111
3,12749.0,2011-12-06 09:56:00,4090.88,199,3,1,1,1,111
4,12820.0,2011-12-06 15:12:00,942.34,59,3,1,2,2,122


TypeError: unhashable type: 'slice'